In [1]:
# Develop by Bolin 
# Date: Aug 16, 2021

In [56]:
import paddle
import paddleseg.transforms as T
from paddleseg.models import BiSeNetV2
from paddleseg.datasets import OpticDiscSeg
from paddleseg.models.losses import CrossEntropyLoss
from paddleseg.core import train
import os

# 模型训练

## 1. 构建模型

In [57]:
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

## 2. 构建训练集及数据预处理流程

### 数据准备

In [58]:
# 构建训练用的数据增强和预处理
transforms = [
    T.Resize(target_size=(100, 100)),
    T.RandomHorizontalFlip(),
    T.Normalize()
]

# 构建训练集
train_dataset = OpticDiscSeg(
    dataset_root='data/outputs',
    transforms=transforms,
    mode='train'
)

### 数据增强

## 3. 构建验证集及数据预处理流程

In [59]:
# 构建验证用的数据增强和预处理
transforms = [
    T.Resize(target_size=(100, 100)),
    T.Normalize()
]

# 构建验证集
val_dataset = OpticDiscSeg(
    dataset_root='data/outputs',
    transforms=transforms,
    mode='val'
)

## 4. 构建优化器

In [60]:
# 设置学习率
base_lr = 0.01
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, power=0.9, decay_steps=1000, end_lr=0)

optimizer = paddle.optimizer.Momentum(lr, parameters=model.parameters(), momentum=0.9, weight_decay=4.0e-5)

## 5. 构建损失函数

In [61]:
losses = {}
losses['types'] = [CrossEntropyLoss()] * 5
losses['coef'] = [1]* 5

## 6. 训练

In [62]:
train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir='output',
    iters=1000,
    batch_size=8,
    save_interval=100,
    log_iters=10,
    num_workers=0,
    losses=losses,
    use_vdl=True)

2021-08-16 00:45:58 [INFO]	[TRAIN] epoch: 1, iter: 10/1000, loss: 2.5609, lr: 0.009919, batch_cost: 0.6814, reader_cost: 0.01088, ips: 11.7412 samples/sec | ETA 00:11:14
2021-08-16 00:46:05 [INFO]	[TRAIN] epoch: 1, iter: 20/1000, loss: 1.2298, lr: 0.009829, batch_cost: 0.7177, reader_cost: 0.00010, ips: 11.1470 samples/sec | ETA 00:11:43
2021-08-16 00:46:13 [INFO]	[TRAIN] epoch: 1, iter: 30/1000, loss: 0.8805, lr: 0.009739, batch_cost: 0.7859, reader_cost: 0.00012, ips: 10.1800 samples/sec | ETA 00:12:42
2021-08-16 00:46:23 [INFO]	[TRAIN] epoch: 1, iter: 40/1000, loss: 0.8304, lr: 0.009648, batch_cost: 1.0464, reader_cost: 0.00023, ips: 7.6454 samples/sec | ETA 00:16:44
2021-08-16 00:46:30 [INFO]	[TRAIN] epoch: 1, iter: 50/1000, loss: 0.7192, lr: 0.009558, batch_cost: 0.6200, reader_cost: 0.00012, ips: 12.9025 samples/sec | ETA 00:09:49
2021-08-16 00:46:37 [INFO]	[TRAIN] epoch: 2, iter: 60/1000, loss: 0.6447, lr: 0.009467, batch_cost: 0.7602, reader_cost: 0.00386, ips: 10.5236 samples/

50/50 [==============================] - 4s 72ms/step - batch_cost: 0.0710 - reader cost: 3.5750e-04


2021-08-16 00:47:08 [INFO]	[EVAL] #Images: 50 mIoU: 0.8845 Acc: 0.9698 Kappa: 0.8736 
2021-08-16 00:47:08 [INFO]	[EVAL] Class IoU: 
[0.9655 0.8035]
2021-08-16 00:47:08 [INFO]	[EVAL] Class Acc: 
[0.9729 0.9492]
2021-08-16 00:47:08 [INFO]	[EVAL] The model with the best validation mIoU (0.8845) was saved at iter 100.
2021-08-16 00:47:17 [INFO]	[TRAIN] epoch: 2, iter: 110/1000, loss: 0.5240, lr: 0.009013, batch_cost: 0.8820, reader_cost: 0.00009, ips: 9.0707 samples/sec | ETA 00:13:04
2021-08-16 00:47:24 [INFO]	[TRAIN] epoch: 3, iter: 120/1000, loss: 0.5020, lr: 0.008922, batch_cost: 0.7263, reader_cost: 0.01246, ips: 11.0146 samples/sec | ETA 00:10:39
2021-08-16 00:47:31 [INFO]	[TRAIN] epoch: 3, iter: 130/1000, loss: 0.4721, lr: 0.008831, batch_cost: 0.7063, reader_cost: 0.00011, ips: 11.3268 samples/sec | ETA 00:10:14
2021-08-16 00:47:37 [INFO]	[TRAIN] epoch: 3, iter: 140/1000, loss: 0.4690, lr: 0.008740, batch_cost: 0.6403, reader_cost: 0.00009, ips: 12.4939 samples/sec | ETA 00:09:10
2

50/50 [==============================] - 3s 58ms/step - batch_cost: 0.0571 - reader cost: 3.4069e-04


2021-08-16 00:48:20 [INFO]	[EVAL] #Images: 50 mIoU: 0.8594 Acc: 0.9642 Kappa: 0.8429 
2021-08-16 00:48:20 [INFO]	[EVAL] Class IoU: 
[0.9596 0.7592]
2021-08-16 00:48:20 [INFO]	[EVAL] Class Acc: 
[0.9615 0.9847]
2021-08-16 00:48:20 [INFO]	[EVAL] The model with the best validation mIoU (0.8845) was saved at iter 100.
2021-08-16 00:48:27 [INFO]	[TRAIN] epoch: 4, iter: 210/1000, loss: 0.4299, lr: 0.008098, batch_cost: 0.6669, reader_cost: 0.00011, ips: 11.9951 samples/sec | ETA 00:08:46
2021-08-16 00:48:34 [INFO]	[TRAIN] epoch: 4, iter: 220/1000, loss: 0.4212, lr: 0.008005, batch_cost: 0.6810, reader_cost: 0.00009, ips: 11.7481 samples/sec | ETA 00:08:51
2021-08-16 00:48:40 [INFO]	[TRAIN] epoch: 5, iter: 230/1000, loss: 0.4001, lr: 0.007913, batch_cost: 0.6273, reader_cost: 0.00409, ips: 12.7532 samples/sec | ETA 00:08:03
2021-08-16 00:48:46 [INFO]	[TRAIN] epoch: 5, iter: 240/1000, loss: 0.4103, lr: 0.007821, batch_cost: 0.6523, reader_cost: 0.00010, ips: 12.2634 samples/sec | ETA 00:08:15


...

2021-08-16 01:01:18 [INFO]	[EVAL] #Images: 50 mIoU: 0.8908 Acc: 0.9722 Kappa: 0.8811 
2021-08-16 01:01:18 [INFO]	[EVAL] Class IoU: 
[0.9684 0.8131]
2021-08-16 01:01:18 [INFO]	[EVAL] Class Acc: 
[0.9701 0.988 ]
2021-08-16 01:01:18 [INFO]	[EVAL] The model with the best validation mIoU (0.9129) was saved at iter 500.
2021-08-16 01:01:29 [INFO]	[TRAIN] epoch: 15, iter: 810/1000, loss: 0.3024, lr: 0.002254, batch_cost: 1.0184, reader_cost: 0.00015, ips: 7.8557 samples/sec | ETA 00:03:13
2021-08-16 01:01:38 [INFO]	[TRAIN] epoch: 15, iter: 820/1000, loss: 0.2878, lr: 0.002147, batch_cost: 0.9924, reader_cost: 0.00012, ips: 8.0610 samples/sec | ETA 00:02:58
2021-08-16 01:01:49 [INFO]	[TRAIN] epoch: 16, iter: 830/1000, loss: 0.3210, lr: 0.002040, batch_cost: 1.0965, reader_cost: 0.00499, ips: 7.2958 samples/sec | ETA 00:03:06
2021-08-16 01:01:59 [INFO]	[TRAIN] epoch: 16, iter: 840/1000, loss: 0.2955, lr: 0.001933, batch_cost: 0.9765, reader_cost: 0.00015, ips: 8.1927 samples/sec | ETA 00:02:36


50/50 [==============================] - 4s 77ms/step - batch_cost: 0.0758 - reader cost: 6.3861e-04: 1s - batch_cost: 0.0756 - reader c


2021-08-16 01:03:09 [INFO]	[EVAL] #Images: 50 mIoU: 0.8958 Acc: 0.9735 Kappa: 0.8870 
2021-08-16 01:03:09 [INFO]	[EVAL] Class IoU: 
[0.9698 0.8218]
2021-08-16 01:03:09 [INFO]	[EVAL] Class Acc: 
[0.9717 0.986 ]
2021-08-16 01:03:09 [INFO]	[EVAL] The model with the best validation mIoU (0.9129) was saved at iter 500.
2021-08-16 01:03:17 [INFO]	[TRAIN] epoch: 17, iter: 910/1000, loss: 0.3187, lr: 0.001156, batch_cost: 0.8549, reader_cost: 0.00010, ips: 9.3576 samples/sec | ETA 00:01:16
2021-08-16 01:03:26 [INFO]	[TRAIN] epoch: 17, iter: 920/1000, loss: 0.3101, lr: 0.001041, batch_cost: 0.8414, reader_cost: 0.00010, ips: 9.5076 samples/sec | ETA 00:01:07
2021-08-16 01:03:34 [INFO]	[TRAIN] epoch: 17, iter: 930/1000, loss: 0.2853, lr: 0.000925, batch_cost: 0.8445, reader_cost: 0.00012, ips: 9.4726 samples/sec | ETA 00:00:59
2021-08-16 01:03:43 [INFO]	[TRAIN] epoch: 18, iter: 940/1000, loss: 0.3070, lr: 0.000807, batch_cost: 0.8722, reader_cost: 0.00462, ips: 9.1726 samples/sec | ETA 00:00:52


50/50 [==============================] - 4s 75ms/step - batch_cost: 0.0736 - reader cost: 4.4107e-04: 2s - batch_c


2021-08-16 01:04:41 [INFO]	[EVAL] #Images: 50 mIoU: 0.9013 Acc: 0.9748 Kappa: 0.8934 
2021-08-16 01:04:41 [INFO]	[EVAL] Class IoU: 
[0.9713 0.8312]
2021-08-16 01:04:41 [INFO]	[EVAL] Class Acc: 
[0.9735 0.9843]
2021-08-16 01:04:41 [INFO]	[EVAL] The model with the best validation mIoU (0.9129) was saved at iter 500.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
Cannot find suitable count function for <class 'paddle.nn.layer.pooling.MaxPool2D'>. Treat it as zero FLOPs.
<class 'paddle.nn.layer.pooling.AdaptiveAvgPool2D'>'s flops has been counted
<class 'paddle.nn.layer.pooling.AvgPool2D'>'s flops has been counted
Cannot find suitable count function for <class 'paddle.nn.layer.activation.Sigmoid'>. Treat it as zero FLOPs.
<class 'paddle.nn.layer.common.Dropout'>'s flops has been counted
Total Flops: 326522850     Total Params: 2328346


# 模型评估

## 6. 构建模型

In [63]:
from paddleseg.models import BiSeNetV2
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

## 7. 加载模型参数

In [64]:
model_path = 'output/best_model/model.pdparams'
if model_path:
    para_state_dict = paddle.load(model_path)
    model.set_dict(para_state_dict)
    print('Loaded trained params of model successfully')
else: 
    raise ValueError('The model_path is wrong: {}'.format(model_path))

Loaded trained params of model successfully


## 8. 构建验证集

In [65]:
# 构建验证用的transforms
transforms = [
    T.Resize(target_size=(100, 100)),
    T.Normalize()
]

# 构建验证集
from paddleseg.datasets import OpticDiscSeg
val_dataset = OpticDiscSeg(
    dataset_root='data/outputs',
    transforms=transforms,
    mode='val'
)

## 9. 评估

In [66]:
from paddleseg.core import evaluate
evaluate(
        model,
        val_dataset)

2021-08-16 01:04:43 [INFO]	Start evaluating (total_samples: 50, total_iters: 50)...


50/50 [==============================] - 4s 78ms/step - batch_cost: 0.0769 - reader cost: 5.0431e-04


2021-08-16 01:04:46 [INFO]	[EVAL] #Images: 50 mIoU: 0.9129 Acc: 0.9774 Kappa: 0.9068 
2021-08-16 01:04:46 [INFO]	[EVAL] Class IoU: 
[0.9741 0.8517]
2021-08-16 01:04:46 [INFO]	[EVAL] Class Acc: 
[0.9798 0.9625]


(0.91287243,
 0.9774376,
 array([0.97407985, 0.85166496], dtype=float32),
 array([0.9797629, 0.9624861], dtype=float32),
 0.9067907145789128)

## 10. 多尺度+翻转评估

In [67]:
evaluate(
        model,
        val_dataset,
        aug_eval=True,
        scales=[0.75, 1.0, 1.25],
        flip_horizontal=True)

2021-08-16 01:04:46 [INFO]	Start evaluating (total_samples: 50, total_iters: 50)...


50/50 [==============================] - 21s 418ms/step - batch_cost: 0.4172 - reader cost: 4.3775e-04


2021-08-16 01:05:07 [INFO]	[EVAL] #Images: 50 mIoU: 0.9235 Acc: 0.9794 Kappa: 0.9189 
2021-08-16 01:05:07 [INFO]	[EVAL] Class IoU: 
[0.9761 0.8709]
2021-08-16 01:05:07 [INFO]	[EVAL] Class Acc: 
[0.9901 0.9194]


(0.9235278,
 0.9794493,
 array([0.9761412 , 0.87091434], dtype=float32),
 array([0.99010384, 0.91944945], dtype=float32),
 0.9189327595036235)

# 效果可视化

## 11. 构建模型

In [68]:
model = BiSeNetV2(num_classes=2,
                 lambd=0.25,
                 align_corners=False,
                 pretrained=None)

## 12. 创建transform

In [69]:
transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.RandomHorizontalFlip(),
    T.Normalize()
])

## 13. 构建待预测的图像列表

In [70]:
def get_image_list(image_path):
    """Get image list"""
    valid_suffix = [
        '.JPEG', '.jpeg', '.JPG', '.jpg', '.BMP', '.bmp', '.PNG', '.png'
    ]
    image_list = []
    image_dir = None
    if os.path.isfile(image_path):
        if os.path.splitext(image_path)[-1] in valid_suffix:
            image_list.append(image_path)
    elif os.path.isdir(image_path):
        image_dir = image_path
        for root, dirs, files in os.walk(image_path):
            for f in files:
                if os.path.splitext(f)[-1] in valid_suffix:
                    image_list.append(os.path.join(root, f))
    else:
        raise FileNotFoundError(
            '`--image_path` is not found. it should be an image file or a directory including images'
        )

    if len(image_list) == 0:
        raise RuntimeError('There are not image file in `--image_path`')

    return image_list, image_dir
image_path = 'data/outputs/JPEGImages/nail501.png' # 也可以输入一个包含图像的目录
image_list, image_dir = get_image_list(image_path)


## 14. 预测

In [71]:
from paddleseg.core import predict
predict(
        model,
        model_path='output/best_model/model.pdparams',
        transforms=transforms,
        image_list=image_list,
        image_dir=image_dir,
        save_dir='output/results'
    )

2021-08-16 01:05:08 [INFO]	Loading pretrained model from output/best_model/model.pdparams
2021-08-16 01:05:08 [INFO]	There are 356/356 variables loaded into BiSeNetV2.
2021-08-16 01:05:08 [INFO]	Start to predict...


1/1 [==============================] - 1s 728ms/step
